In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

In [2]:
data = pd.read_csv(
    "Anket_VeriSeti.csv",
    sep=';',                # Noktalı virgül ayırıcıyı belirt
    encoding='utf-8-sig',   # UTF-8 BOM desteği için
    skip_blank_lines=True   # Boş satırları atla
)

# `\xa0` karakterini temizleme
data = data.apply(lambda x: x.str.strip().replace('\xa0', ' ') if x.dtype == "object" else x)

In [3]:
data.head()

,Id,Cinsiyet,Yas,IlgiAlani,Renk,Butce,Tatil,Dekorasyon,Giyim,Işiklandirma
0,1,Kadın,18-25,Teknoloji ve Cihazlar,Mor,501-1000 TL,Plaj tatilleri,Modern ve minimalist,Rahat ve spor giyim tarzı,Loş ve romantik mum ışığı
1,2,Kadın,18-25,Teknoloji ve Cihazlar,Yeşil,1001-2000 TL,Kültürel ve tarihi mekanları ziyaret etmek,Bohem ve etnik,Klasik ve zarif giyim tarzı,Loş ve romantik mum ışığı
2,3,Erkek,26-35,Teknoloji ve Cihazlar,Gri,5001 TL,Şehir dışı ve uzun yolculuklar,Modern ve minimalist,Rahat ve spor giyim tarzı,Doğal güneş ışığı
3,4,Kadın,18-25,Teknoloji ve Cihazlar,Beyaz,1001-2000 TL,Şehir merkezlerini keşfetmek,Modern ve minimalist,Rahat ve spor giyim tarzı,Yumuşak ve rahatlatıcı LED ışık
4,5,Erkek,18-25,Teknoloji ve Cihazlar,Siyah,1001-2000 TL,"Aktif ve macera dolu tatiller (rafting, sörf, ...",Modern ve minimalist,Rahat ve spor giyim tarzı,Aydınlatılmış şehir manzarası


In [4]:
ilgi_alani_mapping = {
    'Teknoloji ve Cihazlar': 0,
    'Kitaplar ve Edebiyat': 1,
    'Moda ve Giyim': 2,
    'Yemek ve Mutfak': 3,
    'Seyahat ve Macera': 4,
    'Spor ve Aktiviteler': 5,
    'Ev Dekorasyonu': 6,
    'Müzik ve Konserler': 7,
    'Sanat ve El Sanatları': 8
}
data['IlgiAlani'] = data['IlgiAlani'].map(ilgi_alani_mapping)

In [5]:
X = data.drop(['Id', 'IlgiAlani'], axis=1)
y = data['IlgiAlani']

In [6]:
categorical_columns = X.columns

In [7]:
data = data.dropna(subset=['IlgiAlani']) 

In [8]:
categorical_columns = ['Cinsiyet', 'Renk', 'Tatil', 'Dekorasyon', 'Giyim', 'Işiklandirma']

# Pipeline'ı oluşturma
pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)  # Kategorik sütunları dönüştür
        ]
    )),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))  # Modelin kendisi
])

In [9]:
X = data.drop(['Id', 'IlgiAlani'], axis=1)
y = data['IlgiAlani']

# Eğitim ve test setlerine ayırın
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Modeli eğitme
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Cinsiyet', 'Renk', 'Tatil',
                                                   'Dekorasyon', 'Giyim',
                                                   'Işiklandirma'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [11]:
y_pred = pipeline.predict(X_test)

In [12]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Model doğruluğu: {accuracy*100:.2f}%")

Model doğruluğu: 11.43%


In [13]:
new_user = pd.DataFrame({'Cinsiyet': ['Kadın'], 'Yas': [18-25], 'Renk': ['Mavi'], 'Butce': ['1001-2000 TL'], 
                         'Tatil': ['Plaj tatilleri'], 'Dekorasyon': ['Modern ve minimalist'], 'Giyim': ['Rahat ve spor giyim tarzı'], 'Işiklandirma': ['Doğal güneş ışığı']})

In [14]:
print(new_user.isnull().sum())

Cinsiyet        0
Yas             0
Renk            0
Butce           0
Tatil           0
Dekorasyon      0
Giyim           0
Işiklandirma    0
dtype: int64


In [15]:
new_user_pred = pipeline.predict(new_user)
predicted_ilgi_alani = list(ilgi_alani_mapping.keys())[int(new_user_pred[0])]  # float'ı int'e dönüştür
print(f"Yeni kullanıcının ilgi alanı: {predicted_ilgi_alani}")

Yeni kullanıcının ilgi alanı: Seyahat ve Macera


In [16]:
new_user_pred = pipeline.predict(new_user)
predicted_ilgi_alani = list(ilgi_alani_mapping.keys())[int(new_user_pred[0])]  # float'ı int'e dönüştür
print(f"Yeni kullanıcının ilgi alanı: {predicted_ilgi_alani}")

Yeni kullanıcının ilgi alanı: Seyahat ve Macera
